### **Bending Moment and Shear Force Calculator**
(for statically determinate beams only)

**Initialize Containers**

In [1873]:
# DEPENDENCIES
import math #Math functionality
import numpy as np #Numpy for working with arrays 


In [1874]:
#Do not edit - Container initialization
pointLoads = np.array([[]]) # Point Forces [location, xMag, yMag]
pointMoments = np.array([[]])   # Point Moments [locations,Mag]
distributedLoads = np.array([[]]) # Distributed loads [xStart,xEnd,Mag]
linearLoads = np.array([[]]) # linearly varying loads [xStart,xEnd,startMag, endMag] ##note, for this, either startMag or endMag need to be ZERO


**Input Parameters**

In [ ]:
#UNIT SETTINGS___________________ (make sure to stick with these units!)
DIST_UNIT = "ft"
FORCE_UNIT = "kip"
MOMENT_UNIT = FORCE_UNIT + "-" + DIST_UNIT #do not change this! it takes the units and combines them so they can be used for final output

#ANALYSIS PARAMETERS_____________ ## also see unit settings and only use those corresponding units.
span = 8   #Span of beam
A = 0   #Distance to left support
B = .00079  #Distance to right support point

#Force Data______________________ (COMMENT OUT PORTIONS NOT USED) ## also see unit settings and only use those corresponding units.
#pointLoads = np.array([[12,0,-40]]) #[x-dist (+ right), x-force (+ right), y-force (+ up)] #leftmost distace, x-force (left is neg), y-force (down is neg)
#pointMoments = np.array([[17,50]]) #[x-dist, point moment magnitude (+ clockwise)] (use left hand rule)
distributedLoads = np.array([[1,6,-5]]) #[xStart,xEnd,Mag] DOWN FORCES NEGATIVE!
#linearLoads = np.array([[8,17,-10,0]]) # linearly varying loads [xStart,xEnd,startMag, endMag] ##note, for this to work, either startMag or endMag need to be ZERO

set division length and the reaction, shearforce and bendingmoment containers

In [1876]:
#Defaults and initializations
divs = 10000 # divide the span into this number of data points
delta = span / divs #distance between data points
X = np.arange(0,span+delta,delta) #range of x-coordinates
nPL = len(pointLoads[0]) # test for point loads to consider
nPM = len(pointMoments[0]) # test for point moments to consider
nUDL = len(distributedLoads[0]) # test for uniformly distributed loads
nLDL = len(linearLoads[0]) # test for linearly varying magnitude distributed loads

#INITIALISE DATA CONTAINERS
reactions = np.array([0.0,0.0,0.0]) #Reactions (Va, Ha, Vb) - Defined as array of floats to hold reactions
shearForce = np.empty([0,len(X)]) #Shear forces at each data point
bendingMoment = np.empty([0,len(X)]) #Bending moment at each data point


### **Reaction Calculation**

**Define a function to calculate reactions due to point loads**

In [1877]:
def reactions_PL(n):
  xp = pointLoads[n,0] #Location of point load
  fx = pointLoads[n,1] #Point load horizontal component magnitude
  fy = pointLoads[n,2] #Point load vertical component magnitude   

  la_p = A-xp   #Lever arm of point load about point A
  mp = fy*la_p  #Moment generated by point load about A - clockwise moments are positive
  la_vb = B-A   #Lever arm of vertical reaction at B about point A 

  Vb = mp/la_vb #Vertical reaction at B
  Va = -fy-Vb   #Vertical reaction at A  
  Ha = -fx      #Horizontal reaction at A  (assumed to be the pin of the simple support)

  return Va, Vb, Ha

**Define a function to calculate reactions due to point moments**

In [1878]:
def reactions_PM(n):
  xm = pointMoments[n,0] #Location of point load ## looks like this is not needed for now, but could be useful for statically indeterminate beams
  m = pointMoments[n,1] #Point load horizontal component magnitude
  la_vb = B-A

  Vb = m/la_vb #Vertical reaction at B
  Va = -Vb   #Vertical reaction at A  

  return Va, Vb

**Define a function to calculate reactions due to distributed loads**

In [1879]:
def reactions_UDL(n):
  xStart = distributedLoads[n,0]
  xEnd = distributedLoads[n,1]
  fy = distributedLoads[n,2]

  fy_Res = fy*(xEnd - xStart)
  x_Res = xStart + 0.5*(xEnd - xStart)

  la_p = A-x_Res   #Lever arm of resultant point load about point A
  mp = fy_Res*la_p  #Moment generated by resulstant point load about A - clockwise moments are positive
  la_vb = B-A   #Lever arm of vertical reaction at B about point A 

  Vb = mp/la_vb #Vertical reaction at B
  Va = -fy_Res-Vb   #Vertical reaction at A  

  return Va, Vb

**Define a function to calculate reactions due to linearly varying distributed loads**

In [1880]:
def reactions_LDL(n):
    xStart = linearLoads[n,0]
    xEnd = linearLoads[n,1]
    fy_start = linearLoads[n,2]
    fy_end = linearLoads[n,3]

    #Determine the magnitude and location of the force resultant
    if abs(fy_start) > 0:
        fy_Res = 0.5*fy_start*(xEnd - xStart)
        x_Res = xStart + (1/3)*(xEnd-xStart)
    else:
        fy_Res = 0.5*fy_end*(xEnd - xStart)
        x_Res = xStart + (2/3)*(xEnd-xStart)
    
    la_p = A-x_Res   #Lever arm of resultant point load about point A
    mp = fy_Res*la_p  #Moment generated by resulstant point load about A - clockwise moments are positive
    la_vb = B-A   #Lever arm of vertical reaction at B about point A 
    Vb = mp/la_vb #Vertical reaction at B
    Va = -fy_Res-Vb   #Vertical reaction at A  
    
    return Va, Vb

**Cycle through all point loads and determine reactions**

In [1881]:
PL_record = np.empty([0,3])
if(nPL>0):
  for n, p in enumerate(pointLoads):
    va, vb, ha = reactions_PL(n) #Calculate reactions
    PL_record = np.append(PL_record, [np.array([va, ha, vb])], axis=0) #Store reactions for each point load     

    #Add reactions to record (superposition)    
    reactions[0] = reactions[0] + va
    reactions[1] = reactions[1] + ha
    reactions[2] = reactions[2] + vb

**Cycle through all point moments and determine reactions**

In [1882]:
PM_record = np.empty([0,2])
if(nPM>0):
  for n, p in enumerate(pointMoments):
    va, vb = reactions_PM(n) #Calculate reactions
    PM_record = np.append(PM_record, [np.array([va,vb])], axis=0) #Store reactions for each point moment    

    #Add reactions to record (superposition)    
    reactions[0] = reactions[0] + va
    reactions[2] = reactions[2] + vb

**Cycle through all UDLs (uniformly distributed loads) and determine reactions**

In [1883]:
UDL_record = np.empty([0,2])
if(nUDL>0):
  for n, p in enumerate(distributedLoads):
    va, vb = reactions_UDL(n) #Calculate reactions
    UDL_record = np.append(UDL_record, [np.array([va, vb])], axis=0) #Store reactions for each point load     

    #Add reactions to record (superposition)    
    reactions[0] = reactions[0] + va
    reactions[2] = reactions[2] + vb

**Cycle through all LDLs (linearly varying distributed loads) and determine reactions**

In [1884]:
LDL_record = np.empty([0,2])
if(nLDL>0):
  for n, p in enumerate(linearLoads):
    va, vb = reactions_LDL(n) #Calculate reactions
    LDL_record = np.append(LDL_record, [np.array([va, vb])], axis=0) #Store reactions for each point load     

    #Add reactions to record (superposition)    
    reactions[0] = reactions[0] + va
    reactions[2] = reactions[2] + vb

### **Shear and Moment Calculation**

**Define function to calculate shear forces and bending moments due to point loads**

In [1885]:
def shear_moment_PL(n):
  xp = pointLoads[n,0] #Location of point load    
  fy = pointLoads[n,2] #Point load vertical component magnitude     
  Va = PL_record[n,0] #Vertical reaction at A for this point load    
  Vb = PL_record[n,2] #Vertical reaction at B for this point load

  #Cycle through the structure and calculate the shear force and bending moment at each point    
  Shear = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point load    
  Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this point load    
  for i, x in enumerate(X):
    shear = 0 #Initialise the shear force for this data point
    moment = 0 #Initialise the bending moment for this data point 

    if x > A:
      #Calculate shear and moment from reaction at A    
      shear = shear + Va
      moment = moment - Va*(x-A)

    if x > B:
      #Calculate shear and moment from reaction at B    
      shear = shear + Vb
      moment = moment - Vb*(x-B)

    if x > xp:
      #Calculate shear and moment from point load    
      shear = shear + fy
      moment = moment - fy*(x-xp)

    #Store shear and moment for this location
    Shear[i] = shear
    Moment[i] = moment

  return Shear, Moment

**Define function to calculate shear forces and bending moments due to point moments**

In [1886]:
def shear_moment_PM(n):
  xm = pointMoments[n,0] #Location of point moment ## again, this shouldn't affect anything, but the code snippit could be useful later for statically indeterminate beam    
  m = pointMoments[n,1] #Point moment magnitude     
  Va = PM_record[n,0] #Vertical reaction at A for this point moment   
  Vb = PM_record[n,1] #Vertical reaction at B for this point moment

  #Cycle through the structure and calculate the shear force and bending moment at each point    
  Shear = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point moment   
  Moment = np.zeros(len(X)) #Initialise a container to hold all moment force data for this point moment    
  for i, x in enumerate(X):
    shear = 0 #Initialise the shear force for this data point
    moment = 0 #Initialise the bending moment for this data point 

    if x > A:
      #Calculate shear and moment due to reaction at A    
      shear = shear + Va
      moment = moment - Va*(x-A)

    if x > B:
      #Calculate shear and moment due to reaction at B    
      shear = shear + Vb
      moment = moment - Vb*(x-B)

    if x > xm:
      #Calculate moment influence of point moment (no effect on shear)
      moment = moment - m

    #Store shear and moment for this location
    Shear[i] = shear
    Moment[i] = moment

  return Shear, Moment
    

**Define function to calculate shear forces and bending moments due to UDLs**

In [1887]:
def shear_moment_UDL(n):
    xStart = distributedLoads[n,0]    #start location of UDL
    xEnd = distributedLoads[n,1]    #start location of UDL
    fy = distributedLoads[n,2]     #magnitude of UDL
    Va = UDL_record[n,0]
    Vb = UDL_record[n,1]
    

    #cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))    # intitialize a container to hold all the shear force data for this UDL
    Moment = np.zeros(len(X))   # intitialize a container to hold all the bending moment data for this UDL

    for i, x in enumerate(X):
        shear = 0
        moment = 0

        if x > A:
            # calc shear and moment due to reaction at A
            shear = shear + Va
            moment = moment - Va*(x-A)
        
        if x > B:
            # calc shear and moment due to reaction at B
            shear = shear + Vb
            moment = moment - Vb*(x-B)

        if x >= xStart and x <=xEnd:
            # calc shear and moment due to point load
            shear = shear + fy*(x - xStart)
            moment = moment - fy*(x - xStart)*0.5*(x - xStart)
        elif(x>xEnd):
            shear = shear + fy*(xEnd - xStart)
            moment = moment - fy*(xEnd-xStart)*(x-xStart-0.5*(xEnd-xStart))
        
        #store shear and moment for this location
        Shear[i] = shear
        Moment[i] = moment
    
    return Shear, Moment

**Define function to calculate shear forces and bending moments due to LDLs**

In [1888]:
def shear_moment_LDL(n):
     xStart = linearLoads[n,0]   #start location of LDL
     xEnd = linearLoads[n,1]     #start location of LDL
     fy_start = linearLoads[n,2] #magnitude of LDL at start of LDL
     fy_end = linearLoads[n,3]   #magnitude of LDL at end of LDL
     Va = LDL_record[n,0]        #vertical reaction at A for this resultant load
     Vb = LDL_record[n,1]        #vertical reaction at B for this resultant load

     #cycle through the structure and calculate the shear force and bending moment at each point
     Shear = np.zeros(len(X))    # intitialize a container to hold all the shear force data for this LDL
     Moment = np.zeros(len(X))   # intitialize a container to hold all the bending moment data for this LDL
     for i, x in enumerate(X):
          shear = 0
          moment = 0

          if x > A:
               # calc shear and moment due to reaction at A
               shear = shear + Va
               moment = moment - Va*(x-A)
          
          if x > B:
               # calc shear and moment due to reaction at B
               shear = shear + Vb
               moment = moment - Vb*(x-B)
          
          if x>=xStart and x<=xEnd:
               if abs(fy_start)>0:
                    x_base = x - xStart #base of triangle describing linearly dist. load
                    f_cut = fy_start-x_base*(fy_start/(xEnd-xStart)) #starting magnitude of LDL at the cut @ x
                    R1 = x_base*0.5*(fy_start-f_cut)   #mag of resultant due to the triangular component of linear load
                    R2 = x_base*f_cut #mag of reusultant due to the rectangular (constant) portion of the LDL
                    shear = shear + R1 + R2
                    moment = moment - R1*(2/3*x_base) - R2*(x_base/2)

               else:
                    x_base = x - xStart
                    f_cut = fy_end*(x_base/(xEnd-xStart)) 
                    R = .5*f_cut*(x_base)
                    shear = shear + R
                    moment = moment - R*(x_base/3)


          elif x>xEnd:
               if abs(fy_start)>0:
                    R = 0.5*fy_start*(xEnd-xStart)
                    xr = xStart + 1/3*(xEnd-xStart)
                    shear = shear + R
                    moment = moment - R*(x-xr)
               else:
                    R = 0.5*fy_end*(xEnd-xStart)
                    xr = xStart + 2/3*(xEnd-xStart)
                    shear = shear + R
                    moment = moment - R*(x-xr)
                              
          #store shear and moment for this location
          Shear[i] = shear
          Moment[i] = moment

     return Shear, Moment

**Cycle through all point loads and determine shear and moment**

In [1889]:
#Calculate the shear force and bending moment at each datapoint due to point load
if (nPL>0):
  for n, p in enumerate(pointLoads):
    Shear, Moment = shear_moment_PL(n)
    shearForce = np.append(shearForce, [Shear], axis=0) #Store shear force record for each point load
    bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each point load

**Cycle through all point moments and determine shear and moment**

In [1890]:
#Calculate the shear force and bending moment at each datapoint due to point moments
if (nPM>0):
  for n, p in enumerate(pointMoments):
    Shear, Moment = shear_moment_PM(n)
    shearForce = np.append(shearForce, [Shear], axis=0) #Store shear force record for each point load
    bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each point load

**Cycle through all UDLs and determine shear and moment**

In [1891]:
#Calculate the shear force and bending moment at each datapoint due to UDL
if (nUDL>0):
  for n, p in enumerate(distributedLoads):
    Shear, Moment = shear_moment_UDL(n)
    shearForce = np.append(shearForce, [Shear], axis=0) #Store shear force record for each UDL
    bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each UDL

**Cycle through all LDLs and determine shear and moment**

In [1892]:
#Calculate the shear force and bending moment at each datapoint due to LDL
if (nLDL>0):
  for n, p in enumerate(linearLoads):
    Shear, Moment = shear_moment_LDL(n)
    shearForce = np.append(shearForce, [Shear], axis=0) #Store shear force record for each LDL
    bendingMoment = np.append(bendingMoment, [Moment], axis=0) #Store bending moment record for each LDL

### Plotting and Printing

In [1893]:
print(f"The vertical reaction @ A is {reactions[0]:.2f} {FORCE_UNIT}")
print(f"The vertical reaction @ B is {reactions[2]:.2f} {FORCE_UNIT}")
print(f"The horizontal reaction @ A is {reactions[1]:.2f} {FORCE_UNIT}")

The vertical reaction @ A is -110734.49 kip
The vertical reaction @ B is 110759.49 kip
The horizontal reaction @ A is 0.00 kip


### Plotly

In [1894]:
 #Import plotly dependencies...
import plotly as py # import the plotly library
import plotly.graph_objects as go #import the graph object to show the plots
py.offline.init_notebook_mode(connected = True) #Setup offline plotting

In [1895]:
#Define the layout for Shear Diagram
layout = go.Layout(
    title = {
            "text": "Shear Force Diagram",
             "y":0.85,
             "x":0.5,
             "xanchor": "center",
             "yanchor": "top"
             }, 
    yaxis = dict(
        title = f"Shear Force ({FORCE_UNIT})"
        ),
    xaxis = dict(
        title= F"Distance ({DIST_UNIT})",
        range=[-1,span+1],
        ),
showlegend = False,
)

line = go.Scatter(
    x = X,
    y = sum(shearForce),
    mode="lines",
    name="", ## not used. units per below tell the hoverlabel give us enough information already. Use if you want something behind the label
    hovertemplate= "%{y:.1f}" + f" {FORCE_UNIT}", # Note: the .1f part does decimal precision
    fill="tonexty",
    line_color="rgba(255,50,0,1.0)", # rbga(red,green,blue,alpha) where 255 is max value for red,green,blue, and alpha 1 is solid, .1 is transparent
    fillcolor="rgba(255,50,0,0.1)",
)

axis = go.Scatter(
    x = [0,span],
    y = [0,0],
    mode="lines",
    line_color="black"
)

#Generate and view the Figure

fig = go.Figure(data=[line,axis], layout=layout)

py.offline.iplot(fig)



In [1896]:
#Define the layout for Bending Moment Diagram
layout = go.Layout(
    title = {
            "text": "Bending Moment Diagram",
             "y":0.85,
             "x":0.5,
             "xanchor": "center",
             "yanchor": "top"
             }, 
    yaxis = dict(
        title = f"Bending Moment  ({MOMENT_UNIT})",
        autorange="reversed"
        ),
    xaxis = dict(
        title= f"Distance  ({DIST_UNIT})",
        range=[-1,span+1],
        ),
showlegend = False,
)

line = go.Scatter(
    x = X,
    y = -sum(bendingMoment),
    mode="lines",
    name="", ## not used. units tell the hoverlabel what this is already. Use if you want something behind the label
    hovertemplate= "%{y:.1f}" + f" {MOMENT_UNIT}", # Note: the .1f part does decimal precision
    fill="tonexty",
    line_color="rgba(200,0,255,1.0)", # rbga(red,green,blue,alpha) where 255 is max value for red,green,blue, and alpha 1 is solid, .1 is transparent
    fillcolor="rgba(200,0,255,0.1)",
)

axis = go.Scatter(
    x = [0,span],
    y = [0,0],
    mode="lines",
    line_color="black"
)

#Generate and view the Figure

fig = go.Figure(data=[line,axis],layout=layout)

# fig.update_layout(hoverlabel_align = "left") ## not sure what this does. Need to review Plotly documentation

py.offline.iplot(fig)